# Word2Vec Feature Example

In [1]:
import os
import re

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [2]:
DATA_IN_PATH = './data_in/'
TRAIN_CLEAN_DATA = 'train_clean.csv'

RANDOM_SEED = 42
TEST_SPLIT = 0.2

In [3]:
train_data = pd.read_csv(DATA_IN_PATH + TRAIN_CLEAN_DATA)

In [4]:
reviews = list(train_data['review'])
sentiments = list(train_data['sentiment'])

In [5]:
sentences = []
for review in reviews:
    sentences.append(review.split())

In [6]:
num_features = 300    
min_word_count = 40   
num_workers = 4       
context = 10          
downsampling = 1e-3 

In [7]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
   level=logging.INFO)

In [11]:
from gensim.models import word2vec

model = word2vec.Word2Vec(sentences, 
                          workers=num_workers, 
                          vector_size=num_features, # size->vector_size
                          min_count = min_word_count, 
                          window = context, 
                          sample = downsampling)

2021-10-23 14:43:08,465 : INFO : collecting all words and their counts
2021-10-23 14:43:08,468 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-10-23 14:43:08,668 : INFO : PROGRESS: at sentence #10000, processed 1205223 words, keeping 51374 word types
2021-10-23 14:43:08,854 : INFO : PROGRESS: at sentence #20000, processed 2396605 words, keeping 67660 word types
2021-10-23 14:43:08,964 : INFO : collected 74065 word types from a corpus of 2988089 raw words and 25000 sentences
2021-10-23 14:43:08,964 : INFO : Creating a fresh vocabulary
2021-10-23 14:43:09,015 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=40 retains 8160 unique words (11.017349625329103%% of original 74065, drops 65905)', 'datetime': '2021-10-23T14:43:09.015660', 'gensim': '4.1.2', 'python': '3.8.11 (default, Aug  3 2021, 06:49:12) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19042-SP0', 'event': 'prepare_vocab'}
2021-10-23 14:43:09,015 : INFO : Word2Vec lifec

In [12]:
def get_features(words, model, num_features):
    # 출력 벡터 초기화
    feature_vector = np.zeros((num_features),dtype=np.float32)

    num_words = 0
    # 어휘 사전 준비
    index2word_set = set(model.wv.index2word)

    for w in words:
        if w in index2word_set:
            num_words += 1
            # 사전에 해당하는 단어에 대해 단어 벡터를 더함
            feature_vector = np.add(feature_vector, model[w])
    
    # 문장의 단어 수만큼 나누어 단어 벡터의 평균값을 문장 벡터로 함
    feature_vector = np.divide(feature_vector, num_words)
    return feature_vector

In [13]:
def get_dataset(reviews, model, num_features):
    dataset = list()

    for s in reviews:
        dataset.append(get_features(s, model, num_features))

    reviewFeatureVecs = np.stack(dataset)
    
    return reviewFeatureVecs

In [17]:
# test_data_vecs = get_dataset(sentences, model, num_features)

AttributeError: The index2word attribute has been replaced by index_to_key since Gensim 4.0.0.
See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4

In [18]:
from sklearn.model_selection import train_test_split
import numpy as np

# X = test_data_vecs
# y = np.array(sentiments)

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SPLIT, random_state=RANDOM_SEED)

NameError: name 'test_data_vecs' is not defined

In [ ]:
from sklearn.linear_model import LogisticRegression

#lgs = LogisticRegression(class_weight='balanced')
# lgs.fit(X_train, y_train)

In [ ]:
print("Accuracy: %f" % lgs.score(X_test, y_test)) 

In [ ]:
TEST_CLEAN_DATA = 'test_clean.csv'

test_data = pd.read_csv(DATA_IN_PATH + TEST_CLEAN_DATA)

test_review = list(test_data['review'])
test_data.head(5)

In [ ]:
test_sentences = list()
for review in test_review:
    test_sentences.append(review.split())

In [ ]:
test_data_vecs = get_dataset(test_sentences, model, num_features)

In [ ]:
DATA_OUT_PATH = './data_out/'

test_predicted = lgs.predict(test_data_vecs)

if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)
    
ids = list(test_data['id'])
answer_dataset = pd.DataFrame({'id': ids, 'sentiment': test_predicted})
answer_dataset.to_csv(DATA_OUT_PATH + 'lgs_w2v_answer.csv', index=False, quoting=3)

In [ ]:
model_name = "300features_40minwords_10context"
model.save(model_name)